- Dados: Focos de calor do INPE na frequência anual - https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/EstadosBr_sat_ref/MG/

- Dados: Focos de calor do INPE na frequência mensal - https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/

- Código realizado por: Enrique V. Mattos - 11/06/2024

# **1° Passo:** Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime
import os                        # Miscellaneous operating system interfaces
import requests                  # HTTP library for Python
from bs4 import BeautifulSoup    # Pulling data out of HTML and XML files
import re                        # Regular expression operations
import glob                      # Unix style pathname pattern expansion

# vamos ignorar vários avisos
import warnings
warnings.filterwarnings('ignore')

# **PARTE 1):** Baixando e organizando os dados de focos de calor

## - Baixando os dados

In [2]:
%%time

# link dos dados de queimadas do INPE 
url_ano_anteriores = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/' # dados de 2003 - 2023
url_ano_atual = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/' # dados de 2024

#================================================================#
#           ANOS ANTERIORES: focos_br_ref_2003.zip	
#================================================================#
result = requests.get(url_ano_anteriores)                   # Make the connection
soup = BeautifulSoup(result.content, 'html.parser')         # Parse the HTML file
zip_files = soup.find_all('a', href=re.compile("\.zip"))    # Find all "zip" files in the server
filenames = []                                              # Create the list

for i in zip_files:                                         # For each mvol file
    filenames.append(i.extract().get_text())                # Add the file to filenames

    # baixando cada arquivo da lista
    for file in filenames:
        filename = file
        url = f'{url_ano_anteriores}{filename}'
        print("O seguinte arquivo será baixado: ", filename)

        # envia uma requisição para a URL especificada
        myfile = requests.get(url)

        # download o arquivo
        open(f'dados_download_inpe/{filename}', 'wb').write(myfile.content)

#================================================================#
#            ANO ATUAL: focos_mensal_br_202401.csv	
#================================================================#
ano_atual = datetime.now().year
result = requests.get(url_ano_atual)                       # Make the connection
soup = BeautifulSoup(result.content, 'html.parser')        # Parse the HTML file
csv_files = soup.find_all('a', href=re.compile("\.csv"))   # Find all "csv" files in the server
filenames = []                                             # Create the list

for i in csv_files:                                        # For each mvol file
    filenames.append(i.extract().get_text())               # Add the file to filenames

    # baixando cada arquivo da lista
    for file in filenames:
        filename = file
        url = f'{url_ano_atual}{filename}'
        print("O seguinte arquivo será baixado: ", filename)

        # envia uma requisição para a URL especificada
        myfile = requests.get(url)

        # download o arquivo
        open(f'dados_download_inpe/{filename}', 'wb').write(myfile.content)

O seguinte arquivo será baixado:  focos_br_ref_2003.zip
O seguinte arquivo será baixado:  focos_br_ref_2003.zip
O seguinte arquivo será baixado:  focos_br_ref_2004.zip
O seguinte arquivo será baixado:  focos_br_ref_2003.zip
O seguinte arquivo será baixado:  focos_br_ref_2004.zip
O seguinte arquivo será baixado:  focos_br_ref_2005.zip
O seguinte arquivo será baixado:  focos_br_ref_2003.zip
O seguinte arquivo será baixado:  focos_br_ref_2004.zip
O seguinte arquivo será baixado:  focos_br_ref_2005.zip
O seguinte arquivo será baixado:  focos_br_ref_2006.zip
O seguinte arquivo será baixado:  focos_br_ref_2003.zip
O seguinte arquivo será baixado:  focos_br_ref_2004.zip
O seguinte arquivo será baixado:  focos_br_ref_2005.zip
O seguinte arquivo será baixado:  focos_br_ref_2006.zip
O seguinte arquivo será baixado:  focos_br_ref_2007.zip
O seguinte arquivo será baixado:  focos_br_ref_2003.zip
O seguinte arquivo será baixado:  focos_br_ref_2004.zip
O seguinte arquivo será baixado:  focos_br_ref_2

## - Lendo os dados de 2003-2023
- São os dados por `ANO` do `Brasil` do satélite de referência `AQUA`.

In [3]:
%%time

# cria uma tabela (conhecida também com o nome em inglês de "dataframe") vazia
df_2003_a_2023 = pd.DataFrame()

# cria lista dos arquivos
files = sorted(glob.glob('dados_download_inpe/focos_br_ref*'))

# loop de cada arquivo da lista files
for file in files:

    print('Processando ===>>>', file)

    # leitura da tabela
    df0 = pd.read_csv(file, compression='zip')

    # muda o nome da "latitude' para "lat" e "longitude" para "lon" do arquivo de 2023
    if file=='dados_download_inpe\\focos_br_ref_2023.zip':
        df0.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)

    # junta a tabela que foi lida com a anterior
    df_2003_a_2023 = pd.concat([df_2003_a_2023, df0], ignore_index=True)

# remove colunas
df_2003_a_2023.drop(['id_bdq','foco_id','pais'], axis=1, inplace=True)

# renomeia coluna
df_2003_a_2023.rename(columns={'data_pas': 'data'}, inplace=True)

# reposiciona as colunas
df_2003_a_2023 = df_2003_a_2023[['data','lat','lon','municipio','estado','bioma']]

# mostra o dataframe
df_2003_a_2023

Processando ===>>> dados_download_inpe\focos_br_ref_2003.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2004.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2005.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2006.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2007.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2008.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2009.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2010.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2011.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2012.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2013.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2014.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2015.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2016.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2017.zip
Processando ===>>> dados_download_inpe\focos_br_ref_2018.zip
Processando ===>>> dados

,data,lat,lon,municipio,estado,bioma
0,2003-05-15 17:05:00,-18.51800,-55.02800,RIO VERDE DE MATO GROSSO,MATO GROSSO DO SUL,Cerrado
1,2003-05-15 17:05:00,-17.54200,-48.81500,PIRACANJUBA,GOIÁS,Cerrado
2,2003-05-15 17:05:00,-17.61200,-47.16000,GUARDA-MOR,MINAS GERAIS,Cerrado
3,2003-05-15 17:05:00,-18.38600,-51.91900,SERRANÓPOLIS,GOIÁS,Cerrado
4,2003-05-15 17:05:00,-17.87700,-49.09800,MORRINHOS,GOIÁS,Cerrado
...,...,...,...,...,...,...
4829669,2023-07-27 18:00:00,-10.97751,-50.43589,LAGOA DA CONFUSÃO,TOCANTINS,Cerrado
4829670,2023-07-27 18:00:00,-10.97426,-50.44424,LAGOA DA CONFUSÃO,TOCANTINS,Cerrado
4829671,2023-07-27 18:00:00,-10.96047,-50.43910,LAGOA DA CONFUSÃO,TOCANTINS,Cerrado
4829672,2023-07-27 18:00:00,-10.95722,-50.44749,LAGOA DA CONFUSÃO,TOCANTINS,Cerrado


## - Lendo os dados de 2024
- São os dados por `MÊS` do `Brasil` de `TODOS` satélites.

In [4]:
%%time

# cria uma tabela
df_2024 = pd.DataFrame()

# lista os arquivos mensais do ano atual
files = sorted(glob.glob('dados_download_inpe/focos_mensal_br_*csv'))

# loop de cada arquivo da lista files
for file in files:
    
    print('Processando ===>>>', file)
    
    # leitura da tabela
    df0 = pd.read_csv(file, usecols=['lat', 'lon', 'data_hora_gmt', 'satelite', 'municipio', 'estado', 'bioma'])

    # junta a tabela que foi lida com a anterior
    df_2024 = pd.concat([df_2024, df0], ignore_index=True)

# seleciona para o satélite de referência AQUA_M-T
df_2024 = df_2024[df_2024['satelite']=='AQUA_M-T']

# remove colunas
df_2024.drop(['satelite'], axis=1, inplace=True)

# renomeia coluna
df_2024.rename(columns={'data_hora_gmt': 'data'}, inplace=True)

# reposiciona as colunas
df_2024 = df_2024[['data','lat','lon','municipio','estado','bioma']]

# mostra tabela
df_2024

Processando ===>>> dados_download_inpe\focos_mensal_br_202401.csv
Processando ===>>> dados_download_inpe\focos_mensal_br_202402.csv
Processando ===>>> dados_download_inpe\focos_mensal_br_202403.csv
Processando ===>>> dados_download_inpe\focos_mensal_br_202404.csv
Processando ===>>> dados_download_inpe\focos_mensal_br_202405.csv
Processando ===>>> dados_download_inpe\focos_mensal_br_202406.csv
CPU times: total: 1.2 s
Wall time: 2.45 s


,data,lat,lon,municipio,estado,bioma
403,2024-01-01 16:24:00,-12.53837,-41.51642,PALMEIRAS,BAHIA,Caatinga
404,2024-01-01 16:24:00,-12.53544,-41.49259,PALMEIRAS,BAHIA,Caatinga
405,2024-01-01 16:24:00,-7.09835,-42.18570,OEIRAS,PIAUÍ,Caatinga
406,2024-01-01 16:24:00,-12.54596,-41.50789,PALMEIRAS,BAHIA,Caatinga
407,2024-01-01 16:24:00,-12.54324,-41.48442,PALMEIRAS,BAHIA,Caatinga
...,...,...,...,...,...,...
911605,2024-06-24 18:21:00,-8.93617,-61.86473,MACHADINHO D'OESTE,RONDÔNIA,Amazônia
911606,2024-06-24 18:21:00,-8.92855,-61.87508,MACHADINHO D'OESTE,RONDÔNIA,Amazônia
911607,2024-06-24 18:21:00,-8.92725,-61.86605,MACHADINHO D'OESTE,RONDÔNIA,Amazônia
911608,2024-06-24 18:21:00,-8.91963,-61.87640,MACHADINHO D'OESTE,RONDÔNIA,Amazônia


## - Junta os dados de `2003-2023` com `2024`

In [5]:
%%time

# junta os dataframes
df_total = pd.concat([df_2003_a_2023, df_2024], ignore_index=True)

# salva arquivo CSV
df_total.to_csv('dados_download_inpe/focos_br_AQUA_2003_2024.csv', compression={'method': 'zip'})

# mostra o dataframe final
df_total

CPU times: total: 20.8 s
Wall time: 47.9 s


,data,lat,lon,municipio,estado,bioma
0,2003-05-15 17:05:00,-18.51800,-55.02800,RIO VERDE DE MATO GROSSO,MATO GROSSO DO SUL,Cerrado
1,2003-05-15 17:05:00,-17.54200,-48.81500,PIRACANJUBA,GOIÁS,Cerrado
2,2003-05-15 17:05:00,-17.61200,-47.16000,GUARDA-MOR,MINAS GERAIS,Cerrado
3,2003-05-15 17:05:00,-18.38600,-51.91900,SERRANÓPOLIS,GOIÁS,Cerrado
4,2003-05-15 17:05:00,-17.87700,-49.09800,MORRINHOS,GOIÁS,Cerrado
...,...,...,...,...,...,...
4863321,2024-06-24 18:21:00,-8.93617,-61.86473,MACHADINHO D'OESTE,RONDÔNIA,Amazônia
4863322,2024-06-24 18:21:00,-8.92855,-61.87508,MACHADINHO D'OESTE,RONDÔNIA,Amazônia
4863323,2024-06-24 18:21:00,-8.92725,-61.86605,MACHADINHO D'OESTE,RONDÔNIA,Amazônia
4863324,2024-06-24 18:21:00,-8.91963,-61.87640,MACHADINHO D'OESTE,RONDÔNIA,Amazônia


# **PARTE 2):** Criando a tabela de `Estados` X `Anos` 

Criando a coluna ano e agrupando os dados

In [6]:
# leitura do dado
df_total = pd.read_csv('dados_download_inpe/focos_br_AQUA_2003_2024.csv', compression='zip')

# transforma a coluna "datahora" para o formato "datetime"
df_total['data'] = pd.to_datetime(df_total['data'])

# seta a coluna "datahora" como o índice da tabela
df_total.set_index('data', inplace=True)

# cria a coluna ano
df_total['ano'] = df_total.index.year

# agrupa os dados por estado e ano
df_agrupado = df_total.groupby(['estado','ano']).count()['lat']

# mostra os dados
df_total

,Unnamed: 0,lat,lon,municipio,estado,bioma,ano
data,,,,,,,
2003-05-15 17:05:00,0,-18.51800,-55.02800,RIO VERDE DE MATO GROSSO,MATO GROSSO DO SUL,Cerrado,2003
2003-05-15 17:05:00,1,-17.54200,-48.81500,PIRACANJUBA,GOIÁS,Cerrado,2003
2003-05-15 17:05:00,2,-17.61200,-47.16000,GUARDA-MOR,MINAS GERAIS,Cerrado,2003
2003-05-15 17:05:00,3,-18.38600,-51.91900,SERRANÓPOLIS,GOIÁS,Cerrado,2003
2003-05-15 17:05:00,4,-17.87700,-49.09800,MORRINHOS,GOIÁS,Cerrado,2003
...,...,...,...,...,...,...,...
2024-06-24 18:21:00,4863321,-8.93617,-61.86473,MACHADINHO D'OESTE,RONDÔNIA,Amazônia,2024
2024-06-24 18:21:00,4863322,-8.92855,-61.87508,MACHADINHO D'OESTE,RONDÔNIA,Amazônia,2024
2024-06-24 18:21:00,4863323,-8.92725,-61.86605,MACHADINHO D'OESTE,RONDÔNIA,Amazônia,2024


Criando a tabela no formato estados X anos

In [7]:
# nomes dos anos e estados
anos, estados = df_total['ano'].unique(), sorted(df_total['estado'].unique())

# monta novo dataframe de anos x estados
df_estado_ano = pd.DataFrame(index=estados, columns=anos)

# loop nos estados
for estado in estados:
    df_estado_ano.loc[estado] = df_agrupado[estado]

# salva arquivo CSV
df_estado_ano.to_csv('dados_download_inpe/focos_estado_ano.csv')

# mostra a tabela gerada 
df_estado_ano

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
ACRE,10523,7271,15993,6198,8549,5699,3511,8661,3191,4720,...,5779,7684,6295,6626,6802,9193,8828,11840,6562,109
ALAGOAS,373,233,292,268,262,409,360,299,267,377,...,460,388,129,201,232,159,173,140,227,120
AMAPÁ,2516,3413,2020,1665,1484,2153,2456,1000,1396,2518,...,2936,2595,1946,1206,1277,750,676,990,2552,7
AMAZONAS,10191,8083,15644,11697,11293,6701,9280,12139,5028,9114,...,13419,11173,11685,11446,12676,16729,14848,21217,19601,604
BAHIA,19584,18983,18447,10218,29468,20781,8956,17063,13136,16896,...,16513,6311,6450,4956,7371,7912,14359,9820,11917,1638
CEARÁ,11626,10582,8524,6167,7527,7631,5447,4934,4798,4308,...,3158,4316,3486,3034,4304,3979,4379,4136,6808,429
DISTRITO FEDERAL,161,312,140,140,485,201,99,613,270,169,...,155,229,287,88,213,196,259,251,89,31
ESPÍRITO SANTO,976,267,199,431,757,412,239,295,364,384,...,940,599,187,205,622,401,549,474,543,133
GOIÁS,6874,9012,6128,4387,12639,5240,2920,13419,5766,6031,...,6107,4912,6428,3110,7160,6008,6020,4796,3160,1008
MARANHÃO,34548,33585,33691,23302,44765,22122,19132,36277,19315,34299,...,28436,21789,25576,13892,18521,16817,16077,20224,21113,2176


Leitura do arquivo gerado

In [8]:
df = pd.read_csv('dados_download_inpe/focos_estado_ano.csv')
df.rename(columns=({'Unnamed: 0': 'ano'}), inplace=True)
df

,ano,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,ACRE,10523,7271,15993,6198,8549,5699,3511,8661,3191,...,5779,7684,6295,6626,6802,9193,8828,11840,6562,109
1,ALAGOAS,373,233,292,268,262,409,360,299,267,...,460,388,129,201,232,159,173,140,227,120
2,AMAPÁ,2516,3413,2020,1665,1484,2153,2456,1000,1396,...,2936,2595,1946,1206,1277,750,676,990,2552,7
3,AMAZONAS,10191,8083,15644,11697,11293,6701,9280,12139,5028,...,13419,11173,11685,11446,12676,16729,14848,21217,19601,604
4,BAHIA,19584,18983,18447,10218,29468,20781,8956,17063,13136,...,16513,6311,6450,4956,7371,7912,14359,9820,11917,1638
5,CEARÁ,11626,10582,8524,6167,7527,7631,5447,4934,4798,...,3158,4316,3486,3034,4304,3979,4379,4136,6808,429
6,DISTRITO FEDERAL,161,312,140,140,485,201,99,613,270,...,155,229,287,88,213,196,259,251,89,31
7,ESPÍRITO SANTO,976,267,199,431,757,412,239,295,364,...,940,599,187,205,622,401,549,474,543,133
8,GOIÁS,6874,9012,6128,4387,12639,5240,2920,13419,5766,...,6107,4912,6428,3110,7160,6008,6020,4796,3160,1008
9,MARANHÃO,34548,33585,33691,23302,44765,22122,19132,36277,19315,...,28436,21789,25576,13892,18521,16817,16077,20224,21113,2176
